In [1]:
from sklearn.feature_extraction.text import CountVectorizer

from src.data.features import split_train_test, split_features_target, make_document_vectors, make_day_feature
from src.data.features import sentiment_subjectivity
from src.data.features import generate_ents

from src.utils import load_dataset, home

raw = home / 'data' / 'interim'
raw = load_dataset(raw)
comb = raw['combined']
comb = comb.iloc[:, :]

comb.head(2)

dija-labels (1584, 1)
combined (1584, 26)


,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25,final-label
Label,,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,Georgia 'downs two Russian warplanes' as count...,,Russia Today: Columns of troops roll into Sout...,Russian tanks are moving towards the capital o...,"Afghan children raped with 'impunity,' U.N. of...",150 Russian tanks have entered South Ossetia w...,"Breaking: Georgia invades South Ossetia, Russi...",The 'enemy combatent' trials are nothing but a...,Georgian troops retreat from S. Osettain capit...,Did the U.S. Prep Georgia for War with Russia?,,Announcing:Class Action Lawsuit on Behalf of A...,So---Russia and Georgia are at war and the NYT...,China tells Bush to stay out of other countrie...,Did World War III start today?,Georgia Invades South Ossetia - if Russia gets...,Al-Qaeda Faces Islamist Backlash,"Condoleezza Rice: ""The US would not act to pre...",This is a busy day: The European Union has ap...,"Georgia will withdraw 1,000 soldiers from Iraq...",Why the Pentagon Thinks Attacking Iran is a Ba...,Caucasus in crisis: Georgia invades South Ossetia,Indian shoe manufactory - And again in a seri...,Visitors Suffering from Mental Illnesses Banne...,No Help for Mexico's Kidnapping Surge,0.0
1.0,Why wont America and Nato help us? If they won...,Bush puts foot down on Georgian conflict,Jewish Georgian minister: Thanks to Israeli tr...,Georgian army flees in disarray as Russians ad...,Olympic opening ceremony fireworks 'faked,What were the Mossad with fraudulent New Zeala...,Russia angered by Israeli military sale to Geo...,An American citizen living in S.Ossetia blames...,Welcome To World War IV! Now In High Definition!,"Georgia's move, a mistake of monumental propor...",Russia presses deeper into Georgia; U.S. says ...,Abhinav Bindra wins first ever Individual Olym...,U.S. ship heads for Arctic to define territory,Drivers in a Jerusalem taxi station threaten t...,The French Team is Stunned by Phelps and the 4...,Israel and the US behind the Georgian aggression?,"""Do not believe TV, neither Russian nor Georgi...",Riots are still going on in Montreal (Canada) ...,China to overtake US as largest manufacturer,War in South Ossetia [PICS],Israeli Physicians Group Condemns State Torture,Russia has just beaten the United States over...,Perhaps *the* question about the Georgia - Rus...,Russia is so much better at war,,1.0


In [2]:
tr, te = split_train_test(comb)

In [3]:
load = False
x_tr, y_tr = split_features_target(tr, 'train', load=load)
x_te, y_te = split_features_target(te, 'test', load=load)

splitting into x, y train 
cleaning news headlines
target shape (1056, 1) distribution - 0.53125
saving to data/interim
splitting into x, y test 
cleaning news headlines
target shape (528, 1) distribution - 0.5587121212121212
saving to data/interim


## tidf

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from src.models.train import fit, rf_params, lr_params
from sklearn.feature_extraction.text import TfidfVectorizer

rf = fit(
    RandomForestClassifier(**rf_params),
    x_tr, y_tr, x_te, y_te, vec=TfidfVectorizer()
)

tr-score 0.7102272727272727
te-score 0.5549242424242424
avg-te-pred 0.9962121212121212


In [5]:
rf = fit(
    LogisticRegression(**lr_params),
    x_tr, y_tr, x_te, y_te, vec=TfidfVectorizer()
)

tr-score 1.0
te-score 0.5170454545454546
avg-te-pred 0.6553030303030303


## doc2vec

In [6]:
tr_vecs, te_vecs = make_document_vectors(x_tr, x_te)

In [7]:
rf = fit(
    RandomForestClassifier(**rf_params),
    tr_vecs, y_tr, te_vecs, y_te
)

tr-score 0.7433712121212122
te-score 0.5359848484848485
avg-te-pred 0.803030303030303


## time series - day of week

In [8]:
tr_dates, te_dates = make_day_feature(x_tr, x_te)

rf = fit(
    RandomForestClassifier(**rf_params),
    tr_dates, y_tr, te_dates, y_te
)

tr-score 0.5359848484848485
te-score 0.5587121212121212
avg-te-pred 0.9507575757575758


## Senitment & subjectivity

In [9]:
x_s_tr = sentiment_subjectivity(x_tr)
x_s_te = sentiment_subjectivity(x_te)

rf = fit(
    RandomForestClassifier(**rf_params),
    x_s_tr.values, y_tr, x_s_te.values, y_te
)

tr-score 0.6714015151515151
te-score 0.5321969696969697
avg-te-pred 0.8825757575757576


## entity extraction 

In [10]:
tr_ents = generate_ents(x_tr)
te_ents = generate_ents(x_te)

enc = CountVectorizer()
tr_ents = enc.fit_transform(tr_ents)
te_ents = enc.transform(te_ents)

rf = fit(
    RandomForestClassifier(**rf_params),
    tr_ents, y_tr, te_ents, y_te
)

generating tokens
converting ents to str
generating tokens
converting ents to str
tr-score 0.5691287878787878
te-score 0.5587121212121212
avg-te-pred 1.0


In [11]:
x_tr.head()

,news
Label,
0.0,georgia down russian warplane country brink wa...
1.0,will not america nato help will not help help ...
0.0,remember adorable 9-year old sing open ceremon...
0.0,u.s refuse israel weapon attack iran report pr...
1.0,expert admit legalise drug war south osetia 89...


In [12]:
#  tdif
vec = TfidfVectorizer()

tr_f = [
    vec.fit_transform(x_tr), 
]
te_f = [
    vec.transform(x_te), ]